In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import re

In [2]:
books = pd.read_csv('/opt/ml/data/books.csv')
users = pd.read_csv('/opt/ml/data/users.csv')
ratings = pd.read_csv('/opt/ml/data/train_ratings.csv')

#### USER

- age column의 결측치가 약 41%

In [3]:
users.isna().sum()/len(users)

user_id     0.000000
location    0.000000
age         0.408756
dtype: float64

- location 전처리 후 city, state, country로 split

In [4]:
users['location'] = users['location'].str.replace(r'[^0-9a-zA-z:,]', '') # 특수문자 제거
# trans_wrong_loc = {'idaho, idaho' : 'idaho',
#                         'england' : 'united kingdom',
#                         # 'united kingdom, united kingdom' : 'united kingdom',
#                         ',,' : ',',
#                         'fpo, ap' : '',
#                        }
# for key, value in trans_wrong_loc.items():
#     users['location'] = users['location'].str.replace(key, value)

users['location_city'] = users['location'].apply(lambda x : x.split(',')[0].strip()) # split : city, state, country
users['location_state'] = users['location'].apply(lambda x : x.split(',')[1].strip())
users['location_country'] = users['location'].apply(lambda x : x.split(',')[2].strip())

users = users.replace('na', np.nan)
users = users.replace('', np.nan)
users = users.replace('n/a', np.nan)  # location이 n/a, n/a, n/a 로 표시된 경우
users.head()


,user_id,location,age,location_city,location_state,location_country
0,8,"timmins, ontario, canada",NaN,timmins,ontario,canada
1,11400,"ottawa, ontario, canada",49.0,ottawa,ontario,canada
2,11676,"n/a, n/a, n/a",NaN,NaN,NaN,NaN
3,67544,"toronto, ontario, canada",30.0,toronto,ontario,canada
4,85526,"victoria, british columbia, canada",36.0,victoria,british columbia,canada


In [5]:
users[users['location_city'] == '***********']

,user_id,location,age,location_city,location_state,location_country
6150,19842,"***********, illinois, usa",NaN,***********,illinois,usa


- city로 country 찾기

In [6]:
users.isna().sum()

user_id                 0
location                0
age                 27833
location_city         101
location_state       3185
location_country     2122
dtype: int64

In [7]:
modify_location = users[(users['location_country'].isna())&(users['location_city'].notnull())]['location_city'].values
location = users[(users['location'].str.contains('seattle'))&(users['location_country'].notnull())]['location'].value_counts().index[0]

location_list = []
for location in modify_location:
    try:
        right_location = users[(users['location'].str.contains(location))&(users['location_country'].notnull())]['location'].value_counts().index[0]
        location_list.append(right_location)
    except:
        pass

for location in location_list:
    users.loc[users[users['location_city']==location.split(',')[0]].index,'location_state'] = location.split(',')[1]
    users.loc[users[users['location_city']==location.split(',')[0]].index,'location_country'] = location.split(',')[2]

users.isna().sum()

/tmp/ipykernel_42532/3471613607.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  right_location = users[(users['location'].str.contains(location))&(users['location_country'].notnull())]['location'].value_counts().index[0]
/tmp/ipykernel_42532/3471613607.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  right_location = users[(users['location'].str.contains(location))&(users['location_country'].notnull())]['location'].value_counts().index[0]


user_id                 0
location                0
age                 27833
location_city         101
location_state       1091
location_country      273
dtype: int64

In [8]:
users[(users['location_city'].notnull())&(users['location_country'].notnull())&(users['location_state'].isna())]

,user_id,location,age,location_city,location_state,location_country
633,271622,"malmok, n/a, aruba",NaN,malmok,NaN,aruba
867,29259,"mt. airy, , usa",39.0,mt. airy,NaN,usa
911,34994,"brussels, n/a, belgium",54.0,brussels,NaN,belgium
939,38418,"porto alegre, n/a, brazil",25.0,porto alegre,NaN,brazil
1023,50858,"nacogdoches, , usa",19.0,nacogdoches,NaN,usa
...,...,...,...,...,...,...
68007,272154,"macau, n/a, hong kong",37.0,macau,NaN,hong kong
68008,272386,"rotorua, n/a, new zealand",60.0,rotorua,NaN,new zealand
68052,275874,"zürich, n/a, switzerland",NaN,zürich,NaN,switzerland
68064,276729,"rijeka, n/a, croatia",16.0,rijeka,NaN,croatia


- state는 버리는 거 생각해보자

In [9]:
users_drop_state = users.drop(['location_state'], axis=1)
users_drop_state.isna().sum()

user_id                 0
location                0
age                 27833
location_city         101
location_country      273
dtype: int64

In [10]:
users_after_drop = users_drop_state.drop(users_drop_state[users_drop_state['location_city'].isna() & users_drop_state['location_country'].isna()].index) # country, city 둘 다 NaN인거 버리기
users_after_drop

,user_id,location,age,location_city,location_country
0,8,"timmins, ontario, canada",NaN,timmins,canada
1,11400,"ottawa, ontario, canada",49.0,ottawa,canada
3,67544,"toronto, ontario, canada",30.0,toronto,canada
4,85526,"victoria, british columbia, canada",36.0,victoria,canada
5,96054,"ottawa, ontario, canada",29.0,ottawa,canada
...,...,...,...,...,...
68087,278376,"danville, pennsylvania, usa",54.0,danville,usa
68088,278621,"victoria, delaware, canada",74.0,victoria,canada
68089,278636,"irvington, alabama, usa",NaN,irvington,usa
68090,278659,"vancouver, washington, usa",33.0,vancouver,canada


In [11]:
users_after_drop[users_after_drop['location_city'].isna() & users_after_drop['location_country'].isna()] # 얘네는 city나 country 중 하나는 무조건 있음

,user_id,location,age,location_city,location_country


- country는 있고 city는 없는 경우 -> 최빈 도시로 메꾸자
- city는 있고 country는 없는 경우 -> 버리기

In [12]:
users_after_drop[users_after_drop['location_city'].notnull() & users_after_drop['location_country'].isna()] 

,user_id,location,age,location_city,location_country
1879,181316,"managua, ,",36.0,managua,NaN
2020,205019,"aladinma, imo state, n/a, nigeria",29.0,aladinma,NaN
2183,227295,"llanelli, ,",NaN,llanelli,NaN
2683,273110,"arco, ,",51.0,arco,NaN
2743,53456,"ne portland, oregon,",NaN,ne portland,NaN
...,...,...,...,...,...
65073,143210,"orleans, ,",16.0,orleans,NaN
65142,146516,"saronno, lombardia,",31.0,saronno,NaN
65725,164051,"mexico, ,",65.0,mexico,NaN
66518,192259,"grossenseebach, ,",36.0,grossenseebach,NaN


In [13]:
users_after_drop[users_after_drop['location_city'].isna() & users_after_drop['location_country'].notnull()] # 최빈값으로 채우기

,user_id,location,age,location_city,location_country
1008,48630,"n/a, new jersey, usa",34.0,NaN,usa
1065,56399,"n/a, surrey, united kingdom",63.0,NaN,united kingdom
1200,75081,", illinois, usa",29.0,NaN,usa
1590,135778,", massachusetts, usa",45.0,NaN,usa
1819,172293,"n/a, n/a, united kingdom",NaN,NaN,united kingdom
...,...,...,...,...,...
65336,151206,"n/a, california, usa",NaN,NaN,usa
66498,242321,"n/a, sardegna, italy",NaN,NaN,italy
67061,215978,"n/a, antwerp, belgium",55.0,NaN,belgium
67067,216272,", new hampshire, usa",NaN,NaN,usa


In [14]:
country_list = users_after_drop[users_after_drop['location_city'].isna() & users_after_drop['location_country'].notnull()]['location_country'].unique()
country_list

array(['usa', 'united kingdom', 'canada', 'singapore', 'malaysia',
       'hong kong', 'albania', 'new zealand', 'philippines',
       'n/a - on the road', 'japan', 'guernsey', 'denmark', 'australia',
       'luxembourg', 'austria', 'bermuda', 'ireland', 'france', 'kuwait',
       'belize', 'grenada', 'monaco', 'portugal', 'aruba', 'germany',
       'netherlands', 'italy', 'belgium'], dtype=object)

In [15]:
for _country in country_list:
    users_after_drop[users_after_drop['location']==_country] = users_after_drop[users_after_drop['location_country'] == 'usa'].groupby(by='location_city').count().sort_values('location_country', ascending=False).index[0]

In [16]:
users_after_drop.isna().sum()

user_id                 0
location                0
age                 27824
location_city          90
location_country      262
dtype: int64

In [21]:
users_after_drop = users_after_drop.drop(users_after_drop[users_after_drop['location_city'].isna() | users_after_drop['location_country'].isna()].index)
users_after_drop.isna().sum()

user_id                 0
location                0
age                 27634
location_city           0
location_country        0
dtype: int64

#### age를 어떻게 예측할까?
- Users의 ['location_city'], books의 ['isbn', 'year_of_publication', 'category'] 를 한 테이블에 놓고 클러스터링
- 각 클러스터의 age의 평균 or median
- 그러기 위해선 books 먼저 전처리 해주자

### books

In [22]:
books.head()

,isbn,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path
0,0002005018,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,en,['Actresses'],"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg
1,0060973129,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,en,['1940-1949'],"Here, for the first time in paperback, is an o...",images/0060973129.01.THUMBZZZ.jpg
2,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,en,['Medical'],"Describes the great flu epidemic of 1918, an o...",images/0374157065.01.THUMBZZZ.jpg
3,0399135782,The Kitchen God's Wife,Amy Tan,1991.0,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,en,['Fiction'],A Chinese immigrant who is convinced she is dy...,images/0399135782.01.THUMBZZZ.jpg
4,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000.0,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...,en,['History'],"Essays by respected military historians, inclu...",images/0425176428.01.THUMBZZZ.jpg


In [27]:
# 동일 출판사 합치기
books['publisher'].nunique()

11571

In [28]:
publisher_dict=(books['publisher'].value_counts()).to_dict()
publisher_count_df= pd.DataFrame(list(publisher_dict.items()),columns = ['publisher','count'])
publisher_count_df = publisher_count_df.sort_values(by=['count'], ascending = False)
publisher_count_df

,publisher,count
0,Harlequin,3005
1,Ballantine Books,2322
2,Pocket,2274
3,Penguin Books,1943
4,Bantam Books,1938
...,...,...
7371,Carta & the Jerusalem Post,1
7370,Ziba Pub,1
7369,Bedford Books of St. Martin's Press,1
7368,Random House Information Group,1


In [29]:
modify_list = publisher_count_df[publisher_count_df['count']>1].publisher.values
for publisher in modify_list:
    try:
        number = books[books['publisher']==publisher]['isbn'].apply(lambda x: x[:4]).value_counts().index[0]
        right_publisher = books[books['isbn'].apply(lambda x: x[:4])==number]['publisher'].value_counts().index[0]
        books.loc[books[books['isbn'].apply(lambda x: x[:4])==number].index,'publisher'] = right_publisher
    except: 
        pass
books['publisher'].nunique()

1523